In [1]:
import mysql.connector
import pandas as pd
import numpy as np

In [2]:
con = mysql.connector.connect(host='localhost', user='root', password='Pahomov2000+', db='nutritionaldb')

In [3]:
df_complexes = pd.read_sql("SELECT * FROM complexes", con)
df_employees = pd.read_sql("SELECT * FROM employees", con)
df_dishes = pd.read_sql("SELECT * FROM dishes", con)
df_employeeselect = pd.read_sql("SELECT * FROM employeeselect", con)
df_dishesincomplex = pd.read_sql("SELECT * FROM dishesincomplex", con)

In [4]:
df_dishes

,Id,Name
0,1,Азу по-татарски
1,2,Бигос
2,3,Биточек куриный с яйцом и луком
3,4,Биточек мясной с яйцом и луком
4,5,Биточки паровые
...,...,...
293,294,Юшка картофельная с кабачками
294,295,Язык отварной с соусом
295,296,Яйцо столовое 1 кат
296,297,Яйцо отварное


In [5]:
df_dishes['Category'] = 'no'

In [6]:
def fill_cat(df, name):
    df.loc[df['Name'].str.contains(name), ['Category']] = name

In [7]:
unique_names = []

for name in df_dishes['Name']:
    unique_names.append(name.split(' ')[0])

unique_names = sorted(set(unique_names))

for cat in unique_names:
    fill_cat(df_dishes, cat)

In [8]:
df_dishes

,Id,Name,Category
0,1,Азу по-татарски,Азу
1,2,Бигос,Бигос
2,3,Биточек куриный с яйцом и луком,Биточек
3,4,Биточек мясной с яйцом и луком,Биточек
4,5,Биточки паровые,Биточки
...,...,...,...
293,294,Юшка картофельная с кабачками,Юшка
294,295,Язык отварной с соусом,Язык
295,296,Яйцо столовое 1 кат,Яйцо
296,297,Яйцо отварное,Яйцо


In [9]:
df_dishes[df_dishes['Name'] == 'Салат тазалык']

,Id,Name,Category
218,219,Салат тазалык,Салат


In [10]:
def change_cat(df, name, new_name):
    df.loc[df['Name'].str.contains(name), ['Category']] = new_name

In [11]:
df_dishes[df_dishes['Id'] > 180].head(60)

,Id,Name,Category
180,181,Салат из свежих помидоров,Салат
181,182,Салат из свеклы с сыром,Салат
182,183,Салат из свеклы с сыром и чесноком,Салат
183,184,Салат из свеклы с черносливом и орехами,Салат
184,185,Салат из свеклы с чесноком,Салат
185,186,Салат карусель,Салат
186,187,Салат крабик,Салат
187,188,Салат крабовый,Салат
188,189,Салат кудрявый,Салат
189,190,Салат летний по-молдавски,Салат


In [12]:
# Id 1 - 60

change_cat(df_dishes, 'Биточки', 'Биточек')
change_cat(df_dishes, 'Гороховое', 'Гороховое пюре')
change_cat(df_dishes, 'Гороховый', 'Гороховый суп')

change_cat(df_dishes, 'Картофель жареный', 'Картофель жареный')
change_cat(df_dishes, 'Картофель запеченый', 'Картофель запеченый')
change_cat(df_dishes, 'Картофель запеченый', 'Картофель запеченый')
change_cat(df_dishes, 'Картофель отварной', 'Картофель отварной')
change_cat(df_dishes, 'Картофель тушеный в сметаном соусе', 'Картофель тушеный в сметаном соусе')
change_cat(df_dishes, 'Картофель по-селянски', 'Картофель по-селянски')
change_cat(df_dishes, 'Картофельное пюре', 'Картофельное пюре')

change_cat(df_dishes, 'Каша гречневая', 'Каша гречневая')
change_cat(df_dishes, 'Каша гречневая с мясом', 'Каша гречневая с мясом')

In [13]:
# Id 61 - 120

change_cat(df_dishes, 'Каша овсяная', 'Каша овсяная')
change_cat(df_dishes, 'Каша перловая', 'Каша перловая')
change_cat(df_dishes, 'Каша пшеничная', 'Каша пшеничная')
change_cat(df_dishes, 'Каша пшеничная', 'Каша пшеничная')
change_cat(df_dishes, 'Каша пшенная', 'Каша пшенная')
change_cat(df_dishes, 'Каша рисовая', 'Каша рисовая')
change_cat(df_dishes, 'Каша ячневая', 'Каша ячневая')
change_cat(df_dishes, 'Котлета из курицы', 'Котлета куриная')
change_cat(df_dishes, 'Котлета куриная', 'Котлета куриная')
change_cat(df_dishes, 'Котлета рыбная', 'Котлета рыбная')
change_cat(df_dishes, 'Макароны по-флотски', 'Макароны по-флотски')
change_cat(df_dishes, 'Мясное суфле', 'Мясное суфле')

# Категорию "Мясо" рассмотреть отдельно !!!

In [14]:
# Id 121 - 180

change_cat(df_dishes, 'Похлебка по-суворовски', 'Похлебка по-суворовски')

# Salads processing
df_dishes.loc[df_dishes['Name'].str.contains('Сал.'), ['Category']] = df_dishes[df_dishes['Name'].str.contains('Сал.')].Name.values

In [15]:
df_dishes.loc[df_dishes['Name'].str.contains('морков'), ['Category']] = 'Салат из моркови'

In [16]:
df_dishes.loc[df_dishes['Name'].str.contains('Морков') & (df_dishes['Id'] > 190), ['Category']] = 'Салат из моркови'

In [17]:
# Обрабатывать начиная с СУПОВ !!!!

In [18]:
# Extracting actual names with prefix "Сал. " into array
# КОСТЫЛЬ, переписать этот говнокод

arr = df_dishes.loc[df_dishes['Name'].str.contains('Сал. '), ['Name']]['Name']
sal_categories = []

for s in arr:
    cat_arr = s.split(" ")
    cat_arr[0] = 'Салат'
    sal_categories.append(" ".join(cat_arr))

sal_categories

['Салат восторг',
 'Салат из морской капусты с зел. горошком и яйцом',
 'Салат из св.капусты с зеленым горошком',
 'Салат из свежих огурцов',
 'Салат меркурий',
 'Салат огород',
 'Салат посейдон',
 'Салат с яйцом и ветчиной']

In [19]:
# Values with "Сал. " in Category processing
# Id 138 - 145

df_dishes.loc[df_dishes['Category'].str.contains('Сал. '), ['Category']] = sal_categories

In [20]:
# Soup processing

change_cat(df_dishes, 'Суп борщ', 'Суп борщ')
change_cat(df_dishes, 'Суп гороховый', 'Суп гороховый')
change_cat(df_dishes, 'Суп куриный', 'Суп куриный')
change_cat(df_dishes, 'Суп овощной', 'Суп овощной')
change_cat(df_dishes, 'Суп рассольник ленинградский', 'Рассольник')
change_cat(df_dishes, 'Суп картоф', 'Суп картофельный')
change_cat(df_dishes, 'Суп с крупой', 'Суп с крупой')
change_cat(df_dishes, 'Суп тыкмач', 'Суп тыкмач')
change_cat(df_dishes, 'Сырный суп', 'Сырный суп')
change_cat(df_dishes, 'Суп харчо', 'Харчо')

change_cat(df_dishes, 'Тушен', 'Тушеные куриные желудки')
change_cat(df_dishes, 'Хлеб 2 куска', 'Хлеб')
change_cat(df_dishes, 'Шарик куриный с сыром', 'Шарик куриный с сыром')

In [21]:
# Correcting similar dishes title to common

df_dishes.loc[df_dishes['Name'].str.contains('краб') | df_dishes['Name'].str.contains('Краб'), ['Category']] = 'Салат крабовый'
df_dishes.loc[df_dishes['Name'].str.contains('свек') | df_dishes['Name'].str.contains('Свек'), ['Category']] = 'Салат свекольный'
df_dishes.loc[df_dishes['Name'].str.contains('морской капусты') | df_dishes['Name'].str.contains('Морской капусты'), ['Category']] = 'Салат из морской капусты'
df_dishes.loc[df_dishes['Name'].str.contains('помид') & df_dishes['Name'].str.contains('огур'), ['Category']] = 'Салат из огурцов и помидоров'

In [22]:
# Creating dataframe df_categories where I'm mapping Category and Category ID

categories = sorted(set(df_dishes['Category']))
cat_id = np.arange(1, len(categories) + 1)

df_categories = pd.DataFrame({'Id' : cat_id, 'Category' : categories})

In [23]:
df_categories[df_categories['Id'] > 0].head(60)

,Id,Category
0,1,Азу
1,2,Бигос
2,3,Биточек
3,4,Бифштекс
4,5,Борщ
5,6,Бризоль
6,7,Булочка
7,8,Ветчина
8,9,Винегрет
9,10,Говядина


In [24]:
df_dishes[df_dishes['Id'] > 240].head(60)

,Id,Name,Category
240,241,Суп картоф. с мясными фрикад,Суп картофельный
241,242,Суп картоф. с мясными фрикадельками,Суп картофельный
242,243,Суп картофельный с горохом,Суп картофельный
243,244,Суп картофельный с крупой,Суп картофельный
244,245,Суп картофельный с фасолью и мясом птицы,Суп картофельный
245,246,Суп картофельный с яйцом,Суп картофельный
246,247,Суп куриный с рисом и яйцом,Суп куриный
247,248,Суп овощной с мясом птицы,Суп овощной
248,249,Суп рассольник ленинградский,Рассольник
249,250,Суп с крупой и томатами,Суп с крупой


In [25]:
df_dishes[df_dishes['Name'].str.contains("Тушен")]

,Id,Name,Category
257,258,Тушен. курин. желудки,Тушеные куриные желудки
258,259,Тушеные куриные желудки,Тушеные куриные желудки


Салат Крабик == Салат Крабовый == Салат из крабовых палочек с рисом и кукурузой == Салат с крабовыми палочками и кальмарами

Салат из свеклы с сыром = Салат из свеклы с сыром и чесноком = Салат из свеклы с черносливом и орехами = Салат из свеклы с чесноком

Салат из огурцов и помидоров = Салат из помидоров и огурцов

Салат из морской капусты с зел. горошком и яйцом = Салат из морской капусты с кукурузой = Салат из морской капусты с кукурузой и яйцом = Салат из морской капусты с яйцом



In [26]:
# Prepare data for Category ID column

cat_ids = []

for cat in df_dishes['Category']:
    cat_ids.append(df_categories[df_categories['Category'] == cat].Id.values[0])

# Insert prepared data into Category ID

df_dishes['Cat_Id'] = cat_ids

In [28]:
df_dishes.head(60)

,Id,Name,Category,Cat_Id
0,1,Азу по-татарски,Азу,1
1,2,Бигос,Бигос,2
2,3,Биточек куриный с яйцом и луком,Биточек,3
3,4,Биточек мясной с яйцом и луком,Биточек,3
4,5,Биточки паровые,Биточек,3
5,6,Бифштекс запеченый с луком,Бифштекс,4
6,7,Бифштекс с луком,Бифштекс,4
7,8,Бифштекс с томатом,Бифштекс,4
8,9,Бифштекс с яйцом,Бифштекс,4
9,10,Борщ из свежей капусты с картофелем,Борщ,5


In [34]:
df_categories[df_categories['Id'] > 120].head(60)

,Id,Category
120,121,Салат русский
121,122,Салат с курицей и ананасом
122,123,Салат с яйцом и ветчиной
123,124,Салат свекольный
124,125,Салат сельдь под шубой
125,126,Салат слоеный
126,127,Салат смак
127,128,Салат столичный
128,129,Салат студенческий
129,130,Салат тазалык


Невлияющие на выбор категории: Компот, Майонез, Набор (???), Подлива, Сметана, Соус, Сыр, Хлеб, Чай

In [30]:
FORBIDDEN_CATEGORIES = ['Компот', 'Майонез', 'Набор', 'Подлива', 'Сметана', 'Соус', 'Сыр', 'Хлеб', 'Чай']

In [31]:
FORBIDDEN_DISHES_ID = []

for cat in FORBIDDEN_CATEGORIES:
    FORBIDDEN_DISHES_ID += list(df_dishes[df_dishes['Category'] == cat].Id.values)

In [32]:
FORBIDDEN_CATEGORIES_ID = []

for cat in FORBIDDEN_CATEGORIES:
    FORBIDDEN_CATEGORIES_ID += list(df_categories[df_categories['Category'] == cat].Id.values)

FORBIDDEN_CATEGORIES_ID

[40, 46, 53, 62, 141, 143, 153, 161, 162]

In [35]:
df_dishes[df_dishes['Id'] > 60].head(60)

,Id,Name,Category,Cat_Id
60,61,Каша гречневая с овощами,Каша гречневая,29
61,62,Каша овсяная вязкая с маслом,Каша овсяная,31
62,63,Каша перловая,Каша перловая,32
63,64,Каша перловая с овощами,Каша перловая,32
64,65,Каша пшеничная,Каша пшеничная,33
65,66,Каша пшенная вязкая с маслом,Каша пшенная,34
66,67,Каша рисовая вязкая,Каша рисовая,35
67,68,Каша ячневая,Каша ячневая,36
68,69,Кетчуп томатный,Кетчуп,37
69,70,Кисель,Кисель,38


In [36]:
def get_lines(df, ids, col):
    lines = []
    
    for id_ in ids:
        lines += list(df[df[col] == id_].index)
        
    return lines

In [37]:
ids_to_remove = get_lines(df_dishesincomplex, FORBIDDEN_DISHES_ID, col='DishId')
ids_to_remove

[1333,
 1417,
 1473,
 1491,
 1513,
 1524,
 1580,
 1617,
 1708,
 1750,
 1790,
 1802,
 1860,
 1996,
 2032,
 2045,
 2100,
 2198,
 2246,
 2299,
 2397,
 2532,
 2773,
 2794,
 3125,
 3244,
 3321,
 3624,
 3663,
 4061,
 4098,
 4200,
 4360,
 4453,
 4484,
 4504,
 4601,
 4680,
 4853,
 4873,
 5014,
 5223,
 5243,
 5569,
 5627,
 5954,
 6030,
 2,
 33,
 52,
 77,
 90,
 110,
 130,
 137,
 148,
 156,
 169,
 188,
 196,
 227,
 252,
 299,
 307,
 328,
 355,
 370,
 376,
 396,
 415,
 433,
 472,
 491,
 499,
 533,
 541,
 560,
 589,
 608,
 629,
 647,
 685,
 691,
 704,
 713,
 751,
 770,
 783,
 791,
 800,
 804,
 823,
 863,
 883,
 902,
 928,
 940,
 960,
 998,
 1029,
 1055,
 1066,
 1104,
 1122,
 1130,
 1143,
 1164,
 1205,
 1211,
 1223,
 1260,
 1279,
 1287,
 1337,
 1355,
 1363,
 1375,
 1394,
 1413,
 1428,
 1432,
 1451,
 1469,
 1495,
 1508,
 1544,
 1562,
 1570,
 1583,
 1601,
 1620,
 1653,
 1671,
 1699,
 1727,
 1746,
 1766,
 1773,
 1786,
 1805,
 1823,
 1842,
 1850,
 1863,
 1880,
 1888,
 1900,
 1907,
 1937,
 1956,
 1974,
 

In [38]:
dishes_remove = get_lines(df_dishes, FORBIDDEN_DISHES_ID, col='Id')
dishes_remove

[71, 80, 101, 102, 122, 233, 235, 253, 276, 277, 278, 279, 280, 281]

In [52]:
categories_remove = get_lines(df_categories, FORBIDDEN_CATEGORIES_ID, col='Id')
categories_remove

[39, 45, 52, 61, 140, 142, 152, 160, 161]

In [56]:
df_dishesincomplex.drop(ids_to_remove, inplace=True)
df_dishes.drop(dishes_remove, inplace=True)
df_categories.drop(categories_remove, inplace=True)

In [58]:
def max_dishes(df):
    
    lengths = []
    dates = set(df['Date'])
    
    for date in dates:
        for cmp in range(3):
            lengths.append(df[(df['Date'] == date) & (df['CmpxId'] == cmp + 1)].shape[0])
            
    return max(lengths)

In [61]:
def transpose(mat):
    new_mat = []
    
    for i in range(mat.shape[1]):
        dim2 = []
        
        for j in range(mat.shape[2]):
            dim1 = []
            
            for k in range(mat.shape[0]):
                dim1.append(mat[k][i][j])
            
                
            dim2.append(dim1)
            
        new_mat.append(dim2)
    
    return new_mat

In [62]:
def create_result_table(dishes, employees, dish_in_cmp, selected):
    dates = sorted(set(selected['Date']))
    
    MAX_DISHES = max_dishes(dish_in_cmp)
    NUM_OF_CMPX = 3
    
    res_dates = []
    res_emp_id = []
    res_dishes = []
    res_cmpx_id = []
    
    counter = 0
    
    for date in dates:
        
        cmp_for_date = []
        
        for cmp in range(NUM_OF_CMPX):
            tmp_complex = []
            # .DishId replace to .Cat_Id from dishes table
            for dish in dish_in_cmp[(dish_in_cmp['Date'] == date) & (dish_in_cmp['CmpxId'] == cmp+1)].DishId:
                if dishes[dishes['Id'] == dish].Cat_Id.sum() != 0:
                    tmp_complex.append(dishes[dishes['Id'] == dish].Cat_Id.values[0])
                else:
                    tmp_complex.append(0)
            
            # 8 change to the function where I found max num of dishes in result table
            
            if len(tmp_complex) < MAX_DISHES:
                while len(tmp_complex) != MAX_DISHES:
                    tmp_complex.append(0)
            
            tmp_complex = sorted(tmp_complex)
            
            cmp_for_date.append(tmp_complex)
            
        
        for emp in selected[selected['Date'] == date].EmpId:
            res_dates.append(date)
            res_emp_id.append(emp)
            res_cmpx_id.append(selected[(selected['Date'] == date) & (selected['EmpId'] == emp)].CmpxId.values[0])
            res_dishes.append(cmp_for_date)
        
        counter += 1
        print('Processed ' + str(counter) + ' dates')
    
    vocab = {'EmpId' : res_emp_id, 'Date' : res_dates, 'CmpxId' : res_cmpx_id}
    
    # 8 change to the function where I found max num of dishes in result table
    
    res_dishes = transpose(np.array(res_dishes))
    
    for cmp in range(NUM_OF_CMPX):
        for d in range(MAX_DISHES):
            vocab['Dish_' + str(cmp+1) + str(d+1)] = res_dishes[cmp][d]
    
    result = pd.DataFrame(vocab)
    
    return result

In [63]:
table = create_result_table(df_dishes, df_employees, df_dishesincomplex, df_employeeselect)

Processed 1 dates
Processed 2 dates
Processed 3 dates
Processed 4 dates
Processed 5 dates
Processed 6 dates
Processed 7 dates
Processed 8 dates
Processed 9 dates
Processed 10 dates
Processed 11 dates
Processed 12 dates
Processed 13 dates
Processed 14 dates
Processed 15 dates
Processed 16 dates
Processed 17 dates
Processed 18 dates
Processed 19 dates
Processed 20 dates
Processed 21 dates
Processed 22 dates
Processed 23 dates
Processed 24 dates
Processed 25 dates
Processed 26 dates
Processed 27 dates
Processed 28 dates
Processed 29 dates
Processed 30 dates
Processed 31 dates
Processed 32 dates
Processed 33 dates
Processed 34 dates
Processed 35 dates
Processed 36 dates
Processed 37 dates
Processed 38 dates
Processed 39 dates
Processed 40 dates
Processed 41 dates
Processed 42 dates
Processed 43 dates
Processed 44 dates
Processed 45 dates
Processed 46 dates
Processed 47 dates
Processed 48 dates
Processed 49 dates
Processed 50 dates
Processed 51 dates
Processed 52 dates
Processed 53 dates
Pr

In [79]:
table[table.index >= 60].head(60)

,EmpId,Date,CmpxId,Dish_11,Dish_12,Dish_13,Dish_14,Dish_15,Dish_16,Dish_21,...,Dish_23,Dish_24,Dish_25,Dish_26,Dish_31,Dish_32,Dish_33,Dish_34,Dish_35,Dish_36
60,408,2019-03-01,3,0,0,41,67,130,152,0,...,0,10,90,144,0,0,0,4,24,152
61,9,2019-03-04,2,0,0,34,42,152,171,0,...,0,28,117,168,0,0,0,0,61,167
62,17,2019-03-04,2,0,0,34,42,152,171,0,...,0,28,117,168,0,0,0,0,61,167
63,28,2019-03-04,1,0,0,34,42,152,171,0,...,0,28,117,168,0,0,0,0,61,167
64,33,2019-03-04,1,0,0,34,42,152,171,0,...,0,28,117,168,0,0,0,0,61,167
65,35,2019-03-04,3,0,0,34,42,152,171,0,...,0,28,117,168,0,0,0,0,61,167
66,46,2019-03-04,3,0,0,34,42,152,171,0,...,0,28,117,168,0,0,0,0,61,167
67,51,2019-03-04,2,0,0,34,42,152,171,0,...,0,28,117,168,0,0,0,0,61,167
68,52,2019-03-04,2,0,0,34,42,152,171,0,...,0,28,117,168,0,0,0,0,61,167
69,59,2019-03-04,4,0,0,34,42,152,171,0,...,0,28,117,168,0,0,0,0,61,167


In [85]:
table.to_csv("E:\\Data Science\\Datasets\\EatingData\\result_table.csv", index=False)